In [8]:
from ultralytics import YOLO
from PIL import Image
import os
import supervision as sv

In [15]:
MODEL_PATH = "models/"
IMAGE_PATH = "../imageCrawler/imageCrawler/Japanese_clothing_women"
MODEL_VERSION = "v1"

### Load model and predict

In [16]:
model = YOLO(f"{MODEL_PATH}yolo11_{MODEL_VERSION}.pt")
image_files = [os.path.join(IMAGE_PATH, f) for f in os.listdir(IMAGE_PATH) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

In [17]:
#[0,1,2] = [bottom, coat, top]
images = [Image.open(image) for image in image_files]

results = [model.predict(image, conf=0.25)[0] for image in images]

#results[5].boxes.xyxy, results[5].boxes.conf, results[5].boxes.cls


0: 640x640 2 tops, 130.5ms
Speed: 3.7ms preprocess, 130.5ms inference, 6.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bottom, 2 tops, 100.9ms
Speed: 1.7ms preprocess, 100.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x544 1 bottom, 1 top, 86.0ms
Speed: 1.9ms preprocess, 86.0ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 544)

0: 640x544 1 bottom, 1 top, 91.6ms
Speed: 2.5ms preprocess, 91.6ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 544)

0: 640x544 1 bottom, 1 top, 76.6ms
Speed: 1.6ms preprocess, 76.6ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 544)

0: 640x544 1 top, 71.5ms
Speed: 1.6ms preprocess, 71.5ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 544)

0: 640x512 1 coat, 1 top, 71.2ms
Speed: 1.7ms preprocess, 71.2ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 512)

0: 640x640 1 bottom, 1 top, 82.8ms
Speed: 1.6ms preprocess, 82.8ms inference, 0.3ms 

In [18]:
detections = [sv.Detections.from_ultralytics(res) for res in results]
detections

[Detections(xyxy=array([[     120.61,      105.68,       623.9,       677.1],
        [     100.17,      615.56,      591.03,      699.53]], dtype=float32), mask=None, confidence=array([     0.8301,     0.36523], dtype=float32), class_id=array([2, 2]), tracker_id=None, data={'class_name': array(['top', 'top'], dtype='<U3')}, metadata={}),
 Detections(xyxy=array([[     216.65,      345.68,       455.2,      601.86],
        [     180.62,      71.339,      510.48,      582.14],
        [     180.61,      82.824,      510.78,       374.6]], dtype=float32), mask=None, confidence=array([    0.97107,     0.55367,     0.29753], dtype=float32), class_id=array([0, 2, 2]), tracker_id=None, data={'class_name': array(['bottom', 'top', 'top'], dtype='<U6')}, metadata={}),
 Detections(xyxy=array([[     222.43,      380.17,      518.29,      738.43],
        [     264.31,      146.08,      451.67,      485.49]], dtype=float32), mask=None, confidence=array([    0.97842,     0.34805], dtype=float32), c

In [19]:
bounding_box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

annotated_images = []


for i in range(len(detections)):
    annotated_image = images[i].copy()  
    annotated_image = bounding_box_annotator.annotate(annotated_image, detections[i])
    annotated_image = label_annotator.annotate(annotated_image, detections[i])
    annotated_images.append(annotated_image) 


In [8]:
# for img in annotated_images:
#     sv.plot_image(img, size=(5, 5))

### Output images and corresponding labels

In [20]:
labels_dir = "../color_analysis_V2/data/label"
imgs_dir = "../color_analysis_V2/data/image"

os.makedirs(labels_dir, exist_ok=True)
os.makedirs(imgs_dir, exist_ok=True)

for idx, (img, res) in enumerate(zip(images, results)):
    # save img
    img_file = os.path.join(imgs_dir, f"image_{idx}.jpg")
    img.save(img_file)
    
    # save boundaries
    label_file = os.path.join(labels_dir, f"image_{idx}.txt")
    with open(label_file, "w") as f:
        index_record = set()
        
        for box in res.boxes:
            class_index = int(box.cls[0])  # Class ID
            
            if class_index not in index_record: # Not consider those duplicated positions
                index_record.add(class_index)
            else:
                continue
            
            #confidence = float(box.conf[0])  # Confidence score
            x_min, y_min, x_max, y_max = box.xyxy[0].tolist()  # Bounding box
            # Image dimensions (replace these with actual dimensions if known)
            img_width, img_height = images[idx].size
            
            # Normalize bounding box coordinates
            center_x = (x_min + x_max) / 2 / img_width
            center_y = (y_min + y_max) / 2 / img_height
            width = (x_max - x_min) / img_width
            height = (y_max - y_min) / img_height
            
            # YOLO format
            f.write(f"{class_index} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}\n")

### Deploy custom model to roboflow

In [ ]:
# import roboflow

# API_KEY = "fLPmQrLBxWRLefBkSJwy"

# rf = roboflow.Roboflow(api_key=API_KEY)
# project = rf.workspace().project("color-clothes")

# #can specify weights_filename, default is "weights/best.pt"
# version = project.version("1")
# version.deploy("yolov11", "models", "yolo11_v1.pt")


loading Roboflow workspace...
loading Roboflow project...
View the status of your deployment at: https://app.roboflow.com/finalproject-xkzyp/color-clothes/1
Share your model with the world at: https://universe.roboflow.com/finalproject-xkzyp/color-clothes/model/1
